# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [3]:
random.seed(3)

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asaid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preprocessing data

In [10]:
start = time.time()

In [9]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [ ]:
# cosmetics = pd.read_pickle('reviews.pickle')

# cols = ['type', 'onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
#        'date', 'title', 'description', 'geography', 'channel', 'product_id',
#        'rating', 'sentiment']
# reviews = reviews[cols]

# reviews = reviews[reviews['geography']=='USA']
# reviews.drop(columns=['geography'], inplace=True)

# reviews.to_pickle('reviews_filtered.pickle')

In [8]:
# reviews = pd.read_pickle('reviews_filtered.pickle')

In [9]:
# # Creating date columns in the right dtype and dropping the day of the month: 2019-02-24 => 2019-02-01
# reviews.loc[:,'clean_date'] = pd.to_datetime(reviews['date'], errors='coerce')
# if reviews['clean_date'].isna().sum() > 0:
#     print('{} rows have been dropped because the date format is wrong.'.format(reviews['clean_date'].isna().sum()))
#     display(reviews.loc[reviews['clean_date'].isna(), 'date'])
#     reviews = reviews.dropna(subset='date')
# reviews['date'] = reviews['clean_date']  
# reviews = reviews.drop('clean_date', axis=1)
# reviews['date'] = reviews['date'].dt.to_period('m')

# # Checking for missing data (NA => -1)
# if reviews['rating'].isna().sum()>0:
#     print('{} rows are missing ratings'.format(reviews['rating'].isna().sum()))
#     reviews.loc[:,'rating'] = reviews['rating'].fillna(-1).astype(int)

# if reviews['sentiment'].isna().sum()>0:
#     print('{} rows are missing sentiments'.format(reviews['sentiment'].isna().sum()))
#     reviews.loc[:,'sentiment'] = reviews['sentiment'].fillna(-1).astype(int)
    
# # Transforming rating and sentiment to dummy variables (one-hot encoding)
# reviews.loc[:,'sentiment'] = reviews['sentiment'].str.lower()
# reviews.loc[:,'rating'] = reviews['rating'].astype(int)
# reviews = pd.concat([reviews, pd.get_dummies(data=reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)

# # Readding NAs data to ratings
# reviews.loc[reviews['rating']==-1,'rating'] = np.nan
# reviews.loc[reviews['sentiment']==-1,'sentiment'] = np.nan

# # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
# reviews.loc[:,'sentiment'] = reviews['sentiment_positive'] - reviews['sentiment_negative']

# # Aggregating RR data by OnlinePost_ID

# # Creating a column to count the number of statements by review once aggreagtion happens
# reviews['nb_statements'] = reviews['sentiment']

# reviews = reviews.groupby(['type',
#                            'channel',
#                            'source_product_identifier',  
#                            'date',
#                            'onlinepost_id']).agg({
#     'description': lambda x:'. '.join(list(x)),
#     'nb_statements':'count',
#     'rating':'first',
#     'rating_1':'first',
#     'rating_2':'first',
#     'rating_3':'first',
#     'rating_4':'first',
#     'rating_5':'first',
#     'sentiment_negative':'sum',
#     'sentiment_neutral':'sum',
#     'sentiment_positive':'sum',
#     'sentiment':'mean'
# }).reset_index()

# # Normalize the one hot sentiment encoding counts (sentiment_negative, sentiment_neutral, sentiment_positive) by the nb_statement.
# reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']] = reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']].div(reviews['nb_statements'], axis=0)

# reviews.to_pickle('reviews_wrangled.pickle')

In [31]:
# reviews = pd.read_pickle('reviews_wrangled.pickle')

In [88]:
# # Adding product related words to the stop words
# stop_words = stopwords.words('english')
# stop_words.extend(['from'])

# brands = pd.read_csv('elc_brands.csv', encoding='ISO-8859-1')
# catalogue = pd.read_csv('elc_catalogue.csv', encoding='ISO-8859-1')

# stop_words.extend(list(set(str(w).lower() for w in ' '.join(brands['ELC_Brand'].unique().tolist() +
#                                           catalogue['Major_Category'].unique().tolist() +
#                                           catalogue['Application'].unique().tolist() +
#                                           catalogue['Category'].unique().tolist() +
#                                           catalogue['SubCategory'].unique().tolist() 
#                                          ).replace('/', ' ').split())))
# # Cache stop_words into hash
# stop_words = Counter(stop_words)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     for sentence in sentences:
#         doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True) if token not in stop_words]) )
#         yield([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [93]:
# reviews['tokens'] = list(tqdm.tqdm(preprocess(reviews['description'].values.tolist(), stop_words), position=0, leave=True, total=len(reviews)))

100%|█████████████████████████████████████████████████████████████████████| 4636608/4636608 [3:20:10<00:00, 386.04it/s]


In [12]:
# reviews.to_pickle('reviews_wrangled_w_tokens.pickle')

In [25]:
# reviews = pd.read_pickle('reviews_wrangled_w_tokens.pickle')

In [29]:
# reviews.loc[reviews['type']=='Cosmetics','type']='temps'
# reviews.loc[reviews['type']=='Skincare','type']='Cosmetics'
# reviews.loc[reviews['type']=='temps','type']='Skincare'

In [26]:
# rp = pd.read_csv('reviews_products.csv', low_memory=False)
# products = rp[['type', 'channel', 'source_product_identifier', 'product', 'brand_abbrev', 'elc_brand',
#        'brand_score', 'item_description', 'product_score', 'itemid_4',
#        'major_category_id', 'major_category', 'application_id', 'application',
#        'category_id', 'category', 'sub_category_id', 'sub_category']].drop_duplicates()
# products = products.dropna(subset=['elc_brand'], axis=0)

In [39]:
# reviews  = reviews.merge(products, left_on=['type', 'channel', 'source_product_identifier'], right_on=['type', 'channel', 'source_product_identifier'], how='inner')

In [40]:
# reviews.to_pickle('reviews_products.pickle')

In [8]:
reviews = pd.read_pickle('reviews_products.pickle')

In [9]:
reviews

,type,channel,source_product_identifier,date,onlinepost_id,description,nb_statements,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,tokens,product,brand_abbrev,elc_brand,brand_score,item_description,product_score,itemid_4,major_category_id,major_category,application_id,application,category_id,category,sub_category_id,sub_category
0,Skincare,Amazon USA,B00015GYC2,2015-03,OnlinePost_20200301_24654a95-b252-4dee-8dfa-77...,This was very disappointing.. When I received ...,4,1.0,1,0,0,0,0,0.5,0.500000,0.000000,-0.500000,"[disappointing, receive, cleansing, use, night...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
1,Skincare,Amazon USA,B00015GYC2,2017-01,OnlinePost_20200301_84729e90-665d-4108-8ca5-0b...,I have tried many face washes- this is the bes...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[try, many, wash, well, get, dirt, leave, soft]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
2,Skincare,Amazon USA,B00015GYC2,2017-12,OnlinePost_20200301_9c216502-fba9-48a8-95cb-3b...,Fast service.. could find it from my usual ven...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[fast, service, could, find, usual, vendor, fa...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
3,Skincare,Amazon USA,B00015GYC2,2018-01,OnlinePost_20200301_01d7a936-32dd-454d-b4ef-ab...,Five Stars.. great great product.,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[star, great, great, product]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
4,Skincare,Amazon USA,B00015GYC2,2018-07,OnlinePost_20200301_53a09274-200c-47dc-9cca-52...,Order from La Mer only.. Can't go wrong with L...,3,4.0,0,0,0,1,0,0.0,0.666667,0.333333,0.333333,"[order, go, wrong, order, site]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388052,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_184332048,Maybe I got an old one but this thing was awfu...,6,1.0,1,0,0,0,0,0.5,0.166667,0.333333,-0.166667,"[maybe, get, old, thing, awful, bad, batch, wa...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388053,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_185330102,Love most Smashbox products and used the origi...,5,1.0,1,0,0,0,0,0.2,0.400000,0.400000,0.200000,"[love, use, original, year, disappoint, disapp...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388054,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184010998,The Best Primer.. this is literally the best p...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[good, literally, well, ever, use, use, else, ...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388055,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184176048,I didn't wear it for 24 hours but my eyeshadow...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[wear, hour, crease, stay, entire, time, produ...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup 

## Using class

In [49]:
class TopicModeller():
    def __init__(self, df, column):
        self.df = df.reset_index(drop=True)
        self.column = column
        self.docs = df[column].values.tolist()
    
    def compute_coherence_values(self, corpus, id2word, text, k, a, b):

        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=k, 
                                               random_state=3,
                                               chunksize=100,
                                               passes=10,
                                               alpha=a,
                                               eta=b)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')

        return coherence_model_lda.get_coherence()
    
    def train_valid_lda(self,
                        nb_samples: int = 100000,
                        topics_range: list = [3,6,9],
                        alpha: list = [0.1, 0.01, 'symmetric','asymmetric'],
                        beta: list = [0.1, 'symmetric']):
        
        nb_samples = min(nb_samples, len(self.docs))
        
        # Training set
        train = random.sample(self.docs, nb_samples)
        
        # Create Dictionary
        self.id2word = corpora.Dictionary(train)
        
        # Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in train]
        
        # Result dictionary
        self.model_results = {'Topics': [],
                         'Alpha': [],
                         'Beta': [],
                         'Coherence': []
                        }

        # Can take a long time to run
        print('Validating hyperparameters...')
        if 1 == 1:
            pbar = tqdm.tqdm(total=(len(beta)*(len(alpha))*len(topics_range)), position=0, leave=True)
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in extended_alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = self.compute_coherence_values(corpus=self.corpus, id2word=self.id2word, text=train,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        self.model_results['Topics'].append(k)
                        self.model_results['Alpha'].append(a)
                        self.model_results['Beta'].append(b)
                        self.model_results['Coherence'].append(cv)
                        print('nb topics: {}, alpha: {}, beta: {}, coherence: {}'.format(k,a,b,cv))
                        pbar.update(1)
        self.cv_results = pd.DataFrame(self.model_results).sort_values("Coherence", ascending=False)
        self.cv_results.to_csv('lda_tuning_results.csv', index=False)
        pbar.close()
        
        self.best_param = self.cv_results.iloc[0]
        self.best_num_topics = int(self.best_param['Topics'])
        self.best_alpha = self.best_param['Alpha']
        self.best_beta = self.best_param['Beta']
        
        # Build LDA model
        print('Training best model...')
        self.lda_model = gensim.models.LdaMulticore(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.best_num_topics,
                                                   alpha = self.best_alpha,
                                                   eta = self.best_beta,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
        print('Done.')
        
    def get_docs_topics(self):
        # Term Document Frequency
        corpus = [self.id2word.doc2bow(text) for text in self.docs]
        self.output = pd.concat([self.df, pd.DataFrame(gensim.matutils.corpus2csc(self.lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,self.best_num_topics+1)])], axis=1)
        return self.output

    def view(self):
        pyLDAvis.enable_notebook()
        self.LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)

In [50]:
all_lda = TopicModeller(reviews, 'tokens')
all_lda.train_valid_lda()

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Validating hyperparameters...


  4%|███▍                                                                              | 1/24 [01:59<45:57, 119.91s/it]

nb topics: 3, alpha: 0.1, beta: 0.1, coherence: 0.43325442793685914


  8%|██████▊                                                                           | 2/24 [03:43<42:08, 114.94s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [35:28<01:26, 86.29s/it]

nb topics: 9, alpha: 5.555555555555555, beta: 0.1, coherence: 0.428919616965962


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [36:42<00:00, 91.75s/it]

nb topics: 9, alpha: 5.555555555555555, beta: symmetric, coherence: 0.42970242811999493
Training best model...


Done.


In [17]:
def train_display_save(tokens, nb_samples, num_topics, alpha, eta, name):
    nb_samples = min(nb_samples, len(tokens))
        
    # Training set
    train = tokens.sample(nb_samples, random_state=3).values.tolist()

    # Create Dictionary
    id2word = corpora.Dictionary(train)

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in train]
    start=time.time()
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=num_topics,
                                                   alpha =alpha ,
                                                   eta =eta ,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
    pyLDAvis.enable_notebook()
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    display(LDAvis_prepared)
    pyLDAvis.save_html(LDAvis_prepared, name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+'.html')
    lda_model.save(name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics))
    pickle.dump(id2word, open( "id2word_"+ name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+".p", "wb" ))
    print(time.time()-start)
    return lda_model, id2word

In [27]:
lda, id2word = train_display_save(reviews['tokens'], 500000, 8, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.069830  0.028329       1        1  20.729736
2      0.076020  0.204738       2        1  14.929252
1     -0.019557  0.169726       3        1  12.598625
3      0.226216 -0.141346       4        1  11.920701
6     -0.209182 -0.024747       5        1  11.288316
4      0.181769 -0.078030       6        1  11.221599
7     -0.159278  0.068581       7        1   9.064605
5     -0.165817 -0.227250       8        1   8.247169, topic_info=         Term           Freq          Total Category  logprob  loglift
18    product  137623.000000  137623.000000  Default  30.0000  30.0000
28   coverage   54571.000000   54571.000000  Default  29.0000  29.0000
0         day   70739.000000   70739.000000  Default  28.0000  28.0000
31       long   59033.000000   59033.000000  Default  27.0000  27.0000
12       last   57413.000000   57413.000000  Default  26.0000  26.0000
..        ...            ...            ...      ...      ...      ...
33        way    3611.769531   20952.695312   Topic8  -4.9501   0.7372
24       year    3916.060547   33726.265625   Topic8  -4.8692   0.3421
229      find    3793.058350   30760.125000   Topic8  -4.9011   0.4023
84       also    3358.006348   30289.546875   Topic8  -5.0230   0.2958
11         go    3522.748291   79227.164062   Topic8  -4.9751  -0.6178

[513 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.203295  absolutely
345       2  0.651332  absolutely
345       4  0.145315  absolutely
912       7  0.999050  accentuate
2685      5  0.999555      actual
...     ...       ...         ...
24        2  0.306112        year
24        6  0.577740        year
24        8  0.116111        year
803       4  0.999574      yellow
410       2  0.999400       young

[790 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 7, 5, 8, 6])

1591.295062303543


In [28]:
docs = reviews['tokens'].values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [29]:
output = pd.concat([reviews, pd.DataFrame(gensim.matutils.corpus2csc(lda.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,8+1)])], axis=1)

In [33]:
topics = pd.DataFrame(gensim.matutils.corpus2csc(lda.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,8+1)])

In [35]:
output = pd.concat([reviews, topics], axis=1)

In [36]:
output.to_pickle('reviews_w_topics_test.pkl')

In [66]:
topics = lda.print_topics(num_words=6)
for topic in topics:
    print('Topic #{}'.format(topic[0]+1))
    print('Terms: ', topic[1])

Topic #1
Terms:  0.067*"love" + 0.039*"look" + 0.039*"day" + 0.039*"last" + 0.037*"go" + 0.037*"long"
Topic #2
Terms:  0.097*"product" + 0.062*"love" + 0.048*"great" + 0.027*"would" + 0.025*"buy" + 0.024*"good"
Topic #3
Terms:  0.044*"coverage" + 0.035*"look" + 0.028*"oily" + 0.018*"use" + 0.017*"well" + 0.015*"great"
Topic #4
Terms:  0.025*"use" + 0.021*"eyeliner" + 0.016*"try" + 0.015*"apply" + 0.014*"easy" + 0.012*"line"
Topic #5
Terms:  0.037*"use" + 0.028*"product" + 0.020*"worth" + 0.016*"much" + 0.014*"make" + 0.014*"price"
Topic #6
Terms:  0.074*"shade" + 0.044*"color" + 0.035*"perfect" + 0.034*"lipstick" + 0.025*"tone" + 0.023*"beautiful"
Topic #7
Terms:  0.083*"lash" + 0.045*"size" + 0.033*"sample" + 0.031*"full" + 0.030*"small" + 0.027*"clump"
Topic #8
Terms:  0.092*"use" + 0.060*"year" + 0.039*"good" + 0.039*"ever" + 0.038*"try" + 0.031*"well"


In [53]:
train_display_save(reviews['tokens'], 100000, 10, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.015775  0.026031       1        1  18.946630
2     -0.030103  0.043060       2        1  18.354685
1     -0.040408 -0.204844       3        1  12.672782
3     -0.088873 -0.114353       4        1  12.640062
7     -0.208014 -0.058132       5        1   8.604838
4      0.005549 -0.002353       6        1   7.835526
9      0.191441  0.022633       7        1   7.081956
6      0.232644  0.264097       8        1   4.989347
5      0.212271 -0.187710       9        1   4.938052
8     -0.258732  0.211572      10        1   3.936121, topic_info=        Term          Freq         Total Category  logprob  loglift
38      good  23418.000000  23418.000000  Default  30.0000  30.0000
26      love  42729.000000  42729.000000  Default  29.0000  29.0000
14      look  30842.000000  30842.000000  Default  28.0000  28.0000
18   product  35994.000000  35994.000000  Default  27.0000  27.0000
12      last  14363.000000  14363.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
343   smudge    491.089478   2150.818848  Topic10  -4.8473   1.7580
79      make    837.790100  21189.324219  Topic10  -4.3131   0.0045
48     leave    501.655304   2849.332764  Topic10  -4.8260   1.4980
87      come    551.123474   5539.032227  Topic10  -4.7320   0.9274
725      far    467.604431   2625.565430  Topic10  -4.8963   1.5095

[631 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.360088  absolutely
345       3  0.639731  absolutely
825       2  0.998969      absorb
2685      8  0.998396      actual
164       6  0.269007         add
...     ...       ...         ...
161       7  0.069909       would
24        5  0.999866        year
803       7  0.998215      yellow
410       5  0.998824       young
895       2  0.999023        zone

[1071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 8, 5, 10, 7, 6, 9])

262.4947021007538


(<gensim.models.ldamulticore.LdaMulticore at 0x257863106c8>,
 <gensim.corpora.dictionary.Dictionary at 0x25706752c48>)

In [46]:
train_display_save(reviews['tokens'], 100000, 9, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.108469  0.027858       1        1  18.986517
0     -0.090348 -0.051352       2        1  18.467342
1     -0.146977  0.068572       3        1  15.412463
3     -0.046216  0.157018       4        1  13.617984
4     -0.073122  0.131416       5        1   9.666149
7     -0.089786  0.054310       6        1   9.081335
6     -0.029642 -0.216844       7        1   7.010195
5      0.124829 -0.305699       8        1   5.426736
8      0.459732  0.134721       9        1   2.331272, topic_info=        Term          Freq         Total Category  logprob  loglift
18   product  35333.000000  35333.000000  Default  30.0000  30.0000
26      love  42182.000000  42182.000000  Default  29.0000  29.0000
221    shade   8737.000000   8737.000000  Default  28.0000  28.0000
12      last  14425.000000  14425.000000  Default  27.0000  27.0000
14      look  30503.000000  30503.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
144   easily    580.948853   3308.322266   Topic9  -4.1555   2.0192
48     leave    397.035248   2842.556396   Topic9  -4.5361   1.7903
87      come    444.966003   5524.991211   Topic9  -4.4221   1.2397
254    power    302.402161   1385.983398   Topic9  -4.8084   2.2363
746   around    243.452881    435.818787   Topic9  -5.0252   3.1765

[574 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       2  0.300342  absolutely
345       3  0.425160  absolutely
345       6  0.120917  absolutely
345       7  0.135405  absolutely
345       8  0.017831  absolutely
...     ...       ...         ...
161       8  0.008822       would
560       3  0.995683       write
24        6  0.999874        year
410       6  0.999007       young
895       1  0.998908        zone

[950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4, 5, 8, 7, 6, 9])

274.27768540382385


(<gensim.models.ldamulticore.LdaMulticore at 0x25706754ac8>,
 <gensim.corpora.dictionary.Dictionary at 0x257043ad188>)

In [47]:
train_display_save(reviews['tokens'], 100000, 8, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.092199 -0.097565       1        1  22.650215
2      0.010810 -0.070904       2        1  20.165100
1      0.144907 -0.116076       3        1  14.913006
4      0.051774  0.048814       4        1  11.093914
3     -0.048545  0.126471       5        1   9.366911
7      0.230596  0.134537       6        1   9.116807
5     -0.256451  0.209356       7        1   7.106972
6     -0.225289 -0.234634       8        1   5.587078, topic_info=         Term          Freq         Total Category  logprob  loglift
26       love  42092.000000  42092.000000  Default  30.0000  30.0000
18    product  35416.000000  35416.000000  Default  29.0000  29.0000
22        use  40568.000000  40568.000000  Default  28.0000  28.0000
221     shade   8607.000000   8607.000000  Default  27.0000  27.0000
28   coverage  14275.000000  14275.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
17       nice    875.935791   8534.307617   Topic8  -4.6189   0.6082
164       add    602.496033   2582.739990   Topic8  -4.9931   1.4292
87       come    729.511108   5490.858398   Topic8  -4.8018   0.8662
38       good    762.064026  23001.146484   Topic8  -4.7581  -0.5226
13     little    634.246155   8098.402832   Topic8  -4.9417   0.3377

[535 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.554547  absolutely
345       3  0.400349  absolutely
345       6  0.044986  absolutely
164       1  0.241604         add
164       2  0.190108         add
...     ...       ...         ...
161       8  0.002945       would
24        6  0.999841        year
803       2  0.997523      yellow
1482      6  0.995421    youthful
895       2  0.999312        zone

[905 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4, 8, 6, 7])

267.84580969810486


(<gensim.models.ldamulticore.LdaMulticore at 0x255ab71ab08>,
 <gensim.corpora.dictionary.Dictionary at 0x255ab71a048>)

In [48]:
train_display_save(reviews['tokens'], 100000, 7, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.074015 -0.023061       1        1  26.585098
2      0.046374 -0.017499       2        1  20.277082
1      0.139697 -0.118178       3        1  15.966609
3     -0.115413 -0.134715       4        1  12.974427
4     -0.071352 -0.154189       5        1   9.784108
6      0.250161  0.239317       6        1   7.940190
5     -0.323483  0.208326       7        1   6.472491, topic_info=         Term          Freq         Total Category  logprob  loglift
26       love  41876.000000  41876.000000  Default  30.0000  30.0000
28   coverage  14274.000000  14274.000000  Default  29.0000  29.0000
221     shade   8493.000000   8493.000000  Default  28.0000  28.0000
31       long  14564.000000  14564.000000  Default  27.0000  27.0000
12       last  14289.000000  14289.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
325       end    837.538147   2749.110840   Topic7  -4.8108   1.5490
38       good   1641.659546  23040.900391   Topic7  -4.1378   0.0960
101      line    885.327087   4349.250000   Topic7  -4.7553   1.1458
11         go   1155.084351  20142.388672   Topic7  -4.4893  -0.1210
53       well   1004.630249  17624.919922   Topic7  -4.6289  -0.1271

[467 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.807393  absolutely
345       3  0.192399  absolutely
164       1  0.001168         add
164       2  0.112954         add
164       3  0.570223         add
...     ...       ...         ...
24        1  0.459926        year
24        2  0.148776        year
24        4  0.179661        year
24        7  0.211582        year
895       2  0.998287        zone

[781 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5, 7, 6])

253.27026963233948


(<gensim.models.ldamulticore.LdaMulticore at 0x2579d40c408>,
 <gensim.corpora.dictionary.Dictionary at 0x255ab71a288>)

In [44]:
train_display_save(reviews['tokens'], 100000, 6, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.033418 -0.193957       1        1  29.719330
2     -0.018536 -0.121703       2        1  21.610653
4     -0.103952  0.047963       3        1  13.914922
3     -0.134755  0.239468       4        1  13.309616
1     -0.084845 -0.038921       5        1  13.116549
5      0.375505  0.067151       6        1   8.328928, topic_info=         Term          Freq         Total Category  logprob  loglift
26       love  42008.000000  42008.000000  Default  30.0000  30.0000
28   coverage  14241.000000  14241.000000  Default  29.0000  29.0000
221     shade   8564.000000   8564.000000  Default  28.0000  28.0000
22        use  40470.000000  40470.000000  Default  27.0000  27.0000
31       long  14563.000000  14563.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
19   purchase   1304.255859   7325.937988   Topic6  -4.6201   0.7596
165     blend   1050.231201   4374.625977   Topic6  -4.8367   1.0586
229      find   1140.417847   8246.303711   Topic6  -4.7543   0.5071
142      wear   1136.169434  14570.615234   Topic6  -4.7581  -0.0659
70    natural    992.025696   6739.010254   Topic6  -4.8937   0.5695

[411 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.832115  absolutely
345       5  0.163304  absolutely
345       6  0.004536  absolutely
2685      3  0.998062      actual
588       4  0.999101         ago
...     ...       ...         ...
24        1  0.000233        year
24        2  0.000117        year
24        4  0.999622        year
803       2  0.997582      yellow
895       2  0.998759        zone

[682 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 4, 2, 6])

254.05143880844116


(<gensim.models.ldamulticore.LdaMulticore at 0x25785af5348>,
 <gensim.corpora.dictionary.Dictionary at 0x2577aa14d88>)

In [50]:
train_display_save(reviews['tokens'], 100000, 5, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.206343  0.006687       1        1  32.779236
2      0.058548  0.003389       2        1  25.617870
1      0.092065  0.089972       3        1  17.727249
3     -0.106325 -0.242101       4        1  15.558969
4     -0.250631  0.142053       5        1   8.316674, topic_info=        Term          Freq         Total Category  logprob  loglift
26      love  41870.000000  41870.000000  Default  30.0000  30.0000
18   product  35454.000000  35454.000000  Default  29.0000  29.0000
3    perfect  11541.000000  11541.000000  Default  28.0000  28.0000
14      look  30182.000000  30182.000000  Default  27.0000  27.0000
130     oily   8749.000000   8749.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
424      bad    776.409424   2505.107422   Topic5  -5.1373   1.3155
259     take    850.010132   3871.872070   Topic5  -5.0467   0.9707
10       get    919.954773  11695.739258   Topic5  -4.9677  -0.0557
149     even    791.831116   8889.198242   Topic5  -5.1176   0.0687
287      bit    705.333618   4013.223633   Topic5  -5.2333   0.7482

[349 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
588       4  0.999551       ago
54        5  0.993508  allergic
84        1  0.323196      also
84        2  0.433921      also
84        3  0.103412      also
...     ...       ...       ...
24        1  0.514441      year
24        2  0.004351      year
24        4  0.459138      year
24        5  0.022098      year
895       2  0.999284      zone

[557 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])

248.18611550331116


(<gensim.models.ldamulticore.LdaMulticore at 0x2540ad7fb48>,
 <gensim.corpora.dictionary.Dictionary at 0x255ab71ac08>)

In [51]:
train_display_save(reviews['tokens'], 100000, 4, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.177403  0.068064       1        1  39.062622
2     -0.099496  0.051014       2        1  25.019117
1      0.035498 -0.222900       3        1  18.979799
3      0.241401  0.103822       4        1  16.938448, topic_info=         Term          Freq         Total Category  logprob  loglift
26       love  42880.000000  42880.000000  Default  30.0000  30.0000
28   coverage  14453.000000  14453.000000  Default  29.0000  29.0000
130      oily   8978.000000   8978.000000  Default  28.0000  28.0000
58       lash   7080.000000   7080.000000  Default  27.0000  27.0000
12       last  14433.000000  14433.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
129      much   1905.262451   8142.829102   Topic4  -4.9509   0.3231
11         go   2149.636719  20358.025391   Topic4  -4.8302  -0.4726
53       well   2088.878662  17789.099609   Topic4  -4.8589  -0.3664
149      even   1720.319946   9111.265625   Topic4  -5.0530   0.1086
221     shade   1665.506714   8665.141602   Topic4  -5.0854   0.1264

[276 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.969737  absolutely
345       2  0.000274  absolutely
345       3  0.030159  absolutely
2685      4  0.998311      actual
588       4  1.000034         ago
...     ...       ...         ...
908       2  0.999061     wrinkle
24        1  0.406199        year
24        2  0.168794        year
24        4  0.425053        year
895       2  0.999404        zone

[379 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

242.0355236530304


(<gensim.models.ldamulticore.LdaMulticore at 0x2540ab28548>,
 <gensim.corpora.dictionary.Dictionary at 0x256c6639388>)

In [52]:
train_display_save(reviews['tokens'], 100000, 3, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.156689  0.128233       1        1  48.532356
2     -0.082896 -0.157576       2        1  26.259640
1      0.239585  0.029343       3        1  25.208002, topic_info=         Term          Freq         Total Category  logprob  loglift
26       love  42570.000000  42570.000000  Default  30.0000  30.0000
28   coverage  14611.000000  14611.000000  Default  29.0000  29.0000
130      oily   9000.000000   9000.000000  Default  28.0000  28.0000
31       long  14927.000000  14927.000000  Default  27.0000  27.0000
58       lash   7215.000000   7215.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
194     brand   2323.412354   3881.549316   Topic3  -5.1501   0.8648
38       good   3034.366211  23658.742188   Topic3  -4.8831  -0.6757
79       make   2785.091797  21302.449219   Topic3  -4.9688  -0.6565
53       well   2720.304688  17941.187500   Topic3  -4.9924  -0.5083
42       work   2334.662842  12545.731445   Topic3  -5.1453  -0.3035

[216 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
345       1  0.994734  absolutely
345       3  0.005231  absolutely
84        1  0.431484        also
84        2  0.435472        also
84        3  0.133071        also
...     ...       ...         ...
908       2  1.000033     wrinkle
24        1  0.798336        year
24        2  0.196559        year
24        3  0.005089        year
895       2  0.999121        zone

[273 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

223.44409322738647


(<gensim.models.ldamulticore.LdaMulticore at 0x257043ade88>,
 <gensim.corpora.dictionary.Dictionary at 0x257043adf88>)

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

## Splitting by review

In [193]:
bad_reviews['tokens'].sample(100000)

[['use',
  'daily',
  'basis',
  'prefer',
  'bright',
  'color',
  'neutral',
  'send',
  'love',
  'use',
  'multiple',
  'color',
  'mesh',
  'together',
  'make',
  'end',
  'result',
  'look',
  'fantastic'],
 ['purchase',
  'excited',
  'long',
  'eyelash',
  'use',
  'bit',
  'noticed',
  'eyelash',
  'get',
  'short',
  'short',
  'discontinue',
  'use',
  'want',
  'keep',
  'eyelash',
  'leave',
  'caused',
  'lash',
  'fall',
  'devastate'],
 ['work', 'expect', 'use', 'type', 'product', 'often'],
 ['office',
  'rub',
  'completely',
  'worth',
  'disappointed',
  'rub',
  'day',
  'go',
  'feel'],
 ['try',
  'consistency',
  'much',
  'thick',
  'less',
  'creamy',
  'expect',
  'apply',
  'fairly',
  'well',
  'take',
  'forever',
  'blend',
  'often',
  'contour',
  'several',
  'time',
  'meet',
  'standard',
  'convenient',
  'lot',
  'time',
  'morning',
  'see',
  'store',
  'intrigue',
  'buy',
  'right',
  'away',
  'wanted',
  'love',
  'even',
  'love',
  'highligh

In [202]:
good_reviews_6 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='good_reviews')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.002995 -0.104998       1        1  31.419340
1      0.042058 -0.025823       2        1  20.701605
3      0.043155 -0.145603       3        1  16.225155
4      0.028037 -0.064968       4        1  15.581468
5      0.210482  0.225326       5        1   9.391821
2     -0.326726  0.116066       6        1   6.680601, topic_info=               Term          Freq         Total Category  logprob  loglift
4              easy   9442.000000   9442.000000  Default  30.0000  30.0000
83          product  37547.000000  37547.000000  Default  29.0000  29.0000
78             long  12613.000000  12613.000000  Default  28.0000  28.0000
143           apply   9783.000000   9783.000000  Default  27.0000  27.0000
351        coverage   8042.000000   8042.000000  Default  26.0000  26.0000
64             look  25675.000000  25675.000000  Default  25.0000  25.0000
9               use  39868.000000  39868.000000  Default  24.0000  24.0000
6              last  11831.000000  11831.000000  Default  23.0000  23.0000
35           review   5254.000000   5254.000000  Default  22.0000  22.0000
7              love  44605.000000  44605.000000  Default  21.0000  21.0000
291            stay   7100.000000   7100.000000  Default  20.0000  20.0000
49             part   4190.000000   4190.000000  Default  19.0000  19.0000
76             brow   4041.000000   4041.000000  Default  18.0000  18.0000
127            feel  15541.000000  15541.000000  Default  17.0000  17.0000
79          natural   6679.000000   6679.000000  Default  16.0000  16.0000
151            lash   6368.000000   6368.000000  Default  15.0000  15.0000
3               day  14548.000000  14548.000000  Default  14.0000  14.0000
286        eyeliner   3691.000000   3691.000000  Default  13.0000  13.0000
57             ever   5436.000000   5436.000000  Default  12.0000  12.0000
47          collect   3044.000000   3044.000000  Default  11.0000  11.0000
50        promotion   3039.000000   3039.000000  Default  10.0000  10.0000
38             star   6386.000000   6386.000000  Default   9.0000   9.0000
40             time   9029.000000   9029.000000  Default   8.0000   8.0000
227              go  17979.000000  17979.000000  Default   7.0000   7.0000
34          receive   4682.000000   4682.000000  Default   6.0000   6.0000
114             try  13139.000000  13139.000000  Default   5.0000   5.0000
48            color   5610.000000   5610.000000  Default   4.0000   4.0000
318           shade   7228.000000   7228.000000  Default   3.0000   3.0000
218            nice  10818.000000  10818.000000  Default   2.0000   2.0000
70              buy  10928.000000  10928.000000  Default   1.0000   1.0000
38             star   6385.858887   6386.378906   Topic1  -4.2816   1.1577
48            color   5610.172852   5610.692871   Topic1  -4.4112   1.1577
229        lipstick   4197.028320   4197.548828   Topic1  -4.7014   1.1576
141            size   3630.372803   3630.892822   Topic1  -4.8464   1.1576
52           sticky   1775.277588   1775.797729   Topic1  -5.5618   1.1575
342       packaging   1471.028687   1471.551514   Topic1  -5.7498   1.1574
24         gorgeous   1356.870728   1357.390869   Topic1  -5.8306   1.1574
273         quality   1854.279785   1855.100098   Topic1  -5.5182   1.1573
272         package    964.296448    964.816711   Topic1  -6.1721   1.1572
490          travel    913.925842    914.445984   Topic1  -6.2257   1.1572
504            nude    911.101257    911.621338   Topic1  -6.2288   1.1572
734            cute    814.799988    815.320190   Topic1  -6.3405   1.1571
250            gift    799.111633    799.631775   Topic1  -6.3600   1.1571
385            item    780.479431    780.999695   Topic1  -6.3836   1.1571
479             fun    688.043579    688.563782   Topic1  -6.5096   1.1570
463            balm    674.413574    674.933716   Topic1  -6.5296   1.1570


In [208]:
bad_reviews_6 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='bad_reviews')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062826  0.028108       1        1  26.549032
1      0.046510 -0.080030       2        1  24.693363
3     -0.149407 -0.036393       3        1  14.739627
2      0.064451 -0.237212       4        1  14.377544
4      0.320073  0.160640       5        1   9.846596
5     -0.218802  0.164886       6        1   9.793847, topic_info=               Term          Freq         Total Category  logprob  loglift
98             look  24331.000000  24331.000000  Default  30.0000  30.0000
5           product  44837.000000  44837.000000  Default  29.0000  29.0000
68             lash   5694.000000   5694.000000  Default  28.0000  28.0000
102            oily   6842.000000   6842.000000  Default  27.0000  27.0000
19              use  36284.000000  36284.000000  Default  26.0000  26.0000
75            smell   9348.000000   9348.000000  Default  25.0000  25.0000
21              bad  10172.000000  10172.000000  Default  24.0000  24.0000
211         receive   5693.000000   5693.000000  Default  23.0000  23.0000
112            make  19133.000000  19133.000000  Default  22.0000  22.0000
93         coverage   5148.000000   5148.000000  Default  21.0000  21.0000
221          bottle   4003.000000   4003.000000  Default  20.0000  20.0000
149            long   5833.000000   5833.000000  Default  19.0000  19.0000
72            money   6500.000000   6500.000000  Default  18.0000  18.0000
371             old   3888.000000   3888.000000  Default  17.0000  17.0000
163            work  14127.000000  14127.000000  Default  16.0000  16.0000
204           apply  10791.000000  10791.000000  Default  15.0000  15.0000
210           order   3729.000000   3729.000000  Default  14.0000  14.0000
59            shade   6086.000000   6086.000000  Default  13.0000  13.0000
55              buy  15906.000000  15906.000000  Default  12.0000  12.0000
119            last   6052.000000   6052.000000  Default  11.0000  11.0000
29             ever   4057.000000   4057.000000  Default  10.0000  10.0000
89             year   3743.000000   3743.000000  Default   9.0000   9.0000
270            open   2950.000000   2950.000000  Default   8.0000   8.0000
201            hour   5340.000000   5340.000000  Default   7.0000   7.0000
190             day   8836.000000   8836.000000  Default   6.0000   6.0000
77            break   6833.000000   6833.000000  Default   5.0000   5.0000
6            sample   5288.000000   5288.000000  Default   4.0000   4.0000
380        eyeliner   2795.000000   2795.000000  Default   3.0000   3.0000
11             hard   4683.000000   4683.000000  Default   2.0000   2.0000
350            size   2738.000000   2738.000000  Default   1.0000   1.0000
75            smell   9347.821289   9348.329102   Topic1  -3.7824   1.3261
59            shade   6086.040527   6086.548828   Topic1  -4.2116   1.3261
167           color   3924.119141   3924.627441   Topic1  -4.6504   1.3260
267          sticky   3582.069824   3582.578125   Topic1  -4.7416   1.3260
53         lipstick   2922.914307   2923.449219   Topic1  -4.9450   1.3260
406            pink   2558.615479   2559.123779   Topic1  -5.0781   1.3260
161            tone   2546.654297   2547.162598   Topic1  -5.0828   1.3260
447           scent   1952.230591   1952.738525   Topic1  -5.3486   1.3259
86           shadow   1834.631836   1835.139771   Topic1  -5.4107   1.3259
329           weird   1650.804688   1651.386841   Topic1  -5.5163   1.3258
878            fair   1421.106323   1421.614258   Topic1  -5.6661   1.3258
884          orange   1363.821167   1364.329102   Topic1  -5.7073   1.3258
627           light   1316.100098   1316.608032   Topic1  -5.7429   1.3258
401          bright   1309.973267   1310.481201   Topic1  -5.7476   1.3258
1631          taste   1229.956787   1230.464722   Topic1  -5.8106   1.3258
1017         strong   1586.453003   1587.165039   Topic1  -5.5561   1.3257


279.723669052124


In [207]:
neutral_reviews_6 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='neutral_reviews')

KeyboardInterrupt: 

In [185]:
def save_display(self, title):
    p = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
    pyLDAvis.save_html(p, title+'.html')

In [186]:
save_display(good_lda, 'good_reviews_8_topics')

In [187]:
save_display(bad_lda, 'bad_reviews_8_topics')

In [188]:
save_display(neutral_lda, 'neutral_reviews_8_topics')

In [142]:
good_reviews = reviews[reviews['rating']>=4]
neutral_reviews = reviews[(reviews['rating']>2) & (reviews['rating']<4)]
bad_reviews = reviews[reviews['rating']<=2]

In [145]:
print(len(good_reviews))
good_lda = TopicModeller(good_reviews, 'tokens')
good_lda.train_valid_lda()

3605467


  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Validating hyperparameters...


30it [4:37:23, 554.78s/it]                                                                                             


Training best model...
Done.


In [146]:
display(good_lda.cv_results)

,Topics,Alpha,Beta,Coherence
29,9,5.55556,symmetric,0.529431
28,9,5.55556,0.1,0.528639
23,9,asymmetric,symmetric,0.514659
12,6,asymmetric,0.1,0.514239
13,6,asymmetric,symmetric,0.506266
21,9,symmetric,symmetric,0.506242
22,9,asymmetric,0.1,0.504448
17,6,8.33333,symmetric,0.502483
19,9,0.1,symmetric,0.500803
16,6,8.33333,0.1,0.497481


In [147]:
good_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.185515  0.050049       1        1  11.504065
1      0.304863  0.076978       2        1  11.385640
7     -0.238536  0.326510       3        1  11.344358
4      0.079629  0.008155       4        1  11.161737
3     -0.097432  0.012390       5        1  11.056719
5      0.002551 -0.116474       6        1  11.037022
2      0.047078 -0.006545       7        1  10.965193
0     -0.104379 -0.222222       8        1  10.813502
8     -0.179289 -0.128841       9        1  10.731767, topic_info=               Term          Freq         Total Category  logprob  loglift
24             love  88790.000000  88790.000000  Default  30.0000  30.0000
23             look  51784.000000  51784.000000  Default  29.0000  29.0000
66              use  79123.000000  79123.000000  Default  28.0000  28.0000
94             make  34940.000000  34940.000000  Default  27.0000  27.0000
110            good  45432.000000  45432.000000  Default  26.0000  26.0000
44              try  26186.000000  26186.000000  Default  25.0000  25.0000
50            great  61948.000000  61948.000000  Default  24.0000  24.0000
18               go  35680.000000  35680.000000  Default  23.0000  23.0000
120            easy  18865.000000  18865.000000  Default  22.0000  22.0000
55           little  17893.000000  17893.000000  Default  21.0000  21.0000
395        coverage  16709.000000  16709.000000  Default  20.0000  20.0000
20             last  23678.000000  23678.000000  Default  19.0000  19.0000
132           shade  13907.000000  13907.000000  Default  18.0000  18.0000
42             time  17564.000000  17564.000000  Default  17.0000  17.0000
4              nice  21673.000000  21673.000000  Default  16.0000  16.0000
6             smell  13297.000000  13297.000000  Default  15.0000  15.0000
108             day  29804.000000  29804.000000  Default  14.0000  14.0000
138            lash  12834.000000  12834.000000  Default  13.0000  13.0000
3           natural  13358.000000  13358.000000  Default  12.0000  12.0000
31          perfect  21034.000000  21034.000000  Default  11.0000  11.0000
1              feel  30740.000000  30740.000000  Default  10.0000  10.0000
112            stay  14281.000000  14281.000000  Default   9.0000   9.0000
8              well  33300.000000  33300.000000  Default   8.0000   8.0000
104            full  10811.000000  10811.000000  Default   7.0000   7.0000
205       beautiful  10593.000000  10593.000000  Default   6.0000   6.0000
59          product  73749.000000  73749.000000  Default   5.0000   5.0000
16            color  10996.000000  10996.000000  Default   4.0000   4.0000
22             long  24423.000000  24423.000000  Default   3.0000   3.0000
381            oily  10528.000000  10528.000000  Default   2.0000   2.0000
280           super   9240.000000   9240.000000  Default   1.0000   1.0000
127            brow   6466.593262   6467.497559   Topic1  -3.9596   2.1623
907         eyebrow   2419.318604   2420.221436   Topic1  -4.9428   2.1621
198           brush   5595.109375   5597.710449   Topic1  -4.1044   2.1620
1272          shape   1698.358765   1699.261963   Topic1  -5.2966   2.1619
330            fill   1407.639038   1408.541748   Topic1  -5.4844   2.1618
509            lose    498.418243    499.321228   Topic1  -6.5226   2.1607
2579         figure    470.789673    471.693024   Topic1  -6.5796   2.1606
608            grow    460.961426    461.864075   Topic1  -6.6007   2.1605
283           write    458.559326    459.462067   Topic1  -6.6060   2.1605
1468           draw    596.722046    597.982605   Topic1  -6.3426   2.1604
1582          waste    422.471924    423.374542   Topic1  -6.6879   2.1603
868            hair    404.729248    405.631866   Topic1  -6.7308   2.1602
1672         stroke    400.240021    401.142700   Topic1  -6.7420   2.1602
1225         sparse    366.554352    367.456970   Topic1  -6.82

In [156]:
print(len(bad_reviews))
bad_lda = TopicModeller(bad_reviews, 'tokens')
bad_lda.train_valid_lda()
bad_lda.view()

585520


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Validating hyperparameters...


100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [1:07:46<00:00, 254.17s/it]


Training best model...
Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.006124  0.027560       1        1  18.308479
8      0.129373 -0.044485       2        1  16.898638
2      0.106756  0.000642       3        1  12.217730
5      0.169401  0.006427       4        1  10.379268
4      0.029508 -0.104814       5        1   9.579919
7     -0.092060 -0.284646       6        1   9.128814
3      0.150007  0.200029       7        1   8.353272
0     -0.219805  0.103063       8        1   7.695968
6     -0.267054  0.096224       9        1   7.437911, topic_info=               Term          Freq         Total Category  logprob  loglift
58             feel  13697.000000  13697.000000  Default  30.0000  30.0000
194           smell   9083.000000   9083.000000  Default  29.0000  29.0000
94          product  44124.000000  44124.000000  Default  28.0000  28.0000
43             look  24319.000000  24319.000000  Default  27.0000  27.0000
175            work  13544.000000  13544.000000  Default  26.0000  26.0000
211            star   6378.000000   6378.000000  Default  25.0000  25.0000
193           shade   6122.000000   6122.000000  Default  24.0000  24.0000
22             lash   5967.000000   5967.000000  Default  23.0000  23.0000
17              use  35580.000000  35580.000000  Default  22.0000  22.0000
61             make  19080.000000  19080.000000  Default  21.0000  21.0000
63             oily   6924.000000   6924.000000  Default  20.0000  20.0000
137           break   7063.000000   7063.000000  Default  19.0000  19.0000
55           bottle   4065.000000   4065.000000  Default  18.0000  18.0000
19             year   3841.000000   3841.000000  Default  17.0000  17.0000
73             last   6024.000000   6024.000000  Default  16.0000  16.0000
324             old   3752.000000   3752.000000  Default  15.0000  15.0000
54              bad   9586.000000   9586.000000  Default  14.0000  14.0000
535          sample   5109.000000   5109.000000  Default  13.0000  13.0000
339         receive   5518.000000   5518.000000  Default  12.0000  12.0000
8             money   6413.000000   6413.000000  Default  11.0000  11.0000
306        coverage   5009.000000   5009.000000  Default  10.0000  10.0000
147            dark   4342.000000   4342.000000  Default   9.0000   9.0000
507       sensitive   3536.000000   3536.000000  Default   8.0000   8.0000
295           order   3463.000000   3463.000000  Default   7.0000   7.0000
38             come   6960.000000   6960.000000  Default   6.0000   6.0000
354         formula   4943.000000   4943.000000  Default   5.0000   5.0000
84              try  18349.000000  18349.000000  Default   4.0000   4.0000
119            open   3077.000000   3077.000000  Default   3.0000   3.0000
209          sticky   3457.000000   3457.000000  Default   2.0000   2.0000
105          change   3007.000000   3007.000000  Default   1.0000   1.0000
257            brow   1568.154907   1568.976318   Topic1  -5.1921   1.6973
161            pump   1382.272705   1383.094238   Topic1  -5.3183   1.6972
436            hear   1041.041260   1041.868042   Topic1  -5.6018   1.6970
143           other    968.812744    969.634277   Topic1  -5.6737   1.6970
1011            low    692.621948    693.443481   Topic1  -6.0093   1.6966
509          absorb    571.474426    572.295837   Topic1  -6.2016   1.6964
2069          write    522.500793    523.322205   Topic1  -6.2912   1.6962
511     expectation    424.113678    424.935059   Topic1  -6.4998   1.6959
2279            odd    400.522308    401.343750   Topic1  -6.5570   1.6958
1637        include    390.474579    391.296051   Topic1  -6.5824   1.6957
1103           face    387.609955    388.431335   Topic1  -6.5898   1.6957
92          lancome    366.736664    367.558105   Topic1  -6.6451   1.6956
874         visible    366.030914    366.852356   Topic1  -6.6471   1.6956
1181         market    410.493652    411.416962   Topic1  -6.53

In [157]:
display(bad_lda.cv_results)

,Topics,Alpha,Beta,Coherence
10,9,symmetric,0.1,0.543806
13,9,asymmetric,symmetric,0.539759
9,9,0.1,symmetric,0.538955
12,9,asymmetric,0.1,0.538672
15,9,5.55556,symmetric,0.534416
14,9,5.55556,0.1,0.531963
11,9,symmetric,symmetric,0.531838
8,9,0.1,0.1,0.524311
4,3,asymmetric,0.1,0.479731
2,3,symmetric,0.1,0.479557


In [158]:
bad_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.006124  0.027560       1        1  18.307310
8      0.129373 -0.044485       2        1  16.898687
2      0.106756  0.000642       3        1  12.223215
5      0.169401  0.006427       4        1  10.378328
4      0.029508 -0.104814       5        1   9.576618
7     -0.092060 -0.284646       6        1   9.134280
3      0.150007  0.200029       7        1   8.350942
0     -0.219805  0.103063       8        1   7.692784
6     -0.267054  0.096224       9        1   7.437835, topic_info=               Term          Freq         Total Category  logprob  loglift
58             feel  13696.000000  13696.000000  Default  30.0000  30.0000
194           smell   9088.000000   9088.000000  Default  29.0000  29.0000
94          product  44120.000000  44120.000000  Default  28.0000  28.0000
43             look  24317.000000  24317.000000  Default  27.0000  27.0000
175            work  13544.000000  13544.000000  Default  26.0000  26.0000
211            star   6381.000000   6381.000000  Default  25.0000  25.0000
193           shade   6120.000000   6120.000000  Default  24.0000  24.0000
22             lash   5965.000000   5965.000000  Default  23.0000  23.0000
17              use  35581.000000  35581.000000  Default  22.0000  22.0000
61             make  19081.000000  19081.000000  Default  21.0000  21.0000
63             oily   6924.000000   6924.000000  Default  20.0000  20.0000
137           break   7065.000000   7065.000000  Default  19.0000  19.0000
55           bottle   4063.000000   4063.000000  Default  18.0000  18.0000
19             year   3841.000000   3841.000000  Default  17.0000  17.0000
73             last   6026.000000   6026.000000  Default  16.0000  16.0000
324             old   3752.000000   3752.000000  Default  15.0000  15.0000
54              bad   9589.000000   9589.000000  Default  14.0000  14.0000
535          sample   5107.000000   5107.000000  Default  13.0000  13.0000
339         receive   5516.000000   5516.000000  Default  12.0000  12.0000
8             money   6415.000000   6415.000000  Default  11.0000  11.0000
306        coverage   5009.000000   5009.000000  Default  10.0000  10.0000
147            dark   4341.000000   4341.000000  Default   9.0000   9.0000
507       sensitive   3538.000000   3538.000000  Default   8.0000   8.0000
295           order   3462.000000   3462.000000  Default   7.0000   7.0000
38             come   6959.000000   6959.000000  Default   6.0000   6.0000
354         formula   4943.000000   4943.000000  Default   5.0000   5.0000
84              try  18348.000000  18348.000000  Default   4.0000   4.0000
119            open   3076.000000   3076.000000  Default   3.0000   3.0000
209          sticky   3457.000000   3457.000000  Default   2.0000   2.0000
105          change   3007.000000   3007.000000  Default   1.0000   1.0000
257            brow   1568.053223   1568.874634   Topic1  -5.1921   1.6973
161            pump   1382.183105   1383.004395   Topic1  -5.3183   1.6973
436            hear   1040.973755   1041.800415   Topic1  -5.6018   1.6971
143           other    968.749939    969.571411   Topic1  -5.6737   1.6970
1011            low    692.577026    693.398438   Topic1  -6.0093   1.6967
509          absorb    571.437378    572.258850   Topic1  -6.2016   1.6964
2069          write    522.466919    523.288330   Topic1  -6.2912   1.6963
511     expectation    424.086182    424.907593   Topic1  -6.4998   1.6959
2279            odd    400.496338    401.317810   Topic1  -6.5570   1.6958
1637        include    390.449280    391.270782   Topic1  -6.5824   1.6958
1103           face    387.584839    388.406250   Topic1  -6.5898   1.6958
92          lancome    366.712891    367.534332   Topic1  -6.6451   1.6956
874         visible    366.007172    366.828583   Topic1  -6.6471   1.6956
1181         market    410.467041    411.390350   Topic1  -6.53

In [159]:
print(len(neutral_reviews))
neutral_lda = TopicModeller(neutral_reviews, 'tokens')
neutral_lda.train_valid_lda()
neutral_lda.view()

445621


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Validating hyperparameters...


100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [56:47<00:00, 212.99s/it]


Training best model...
Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062524  0.067366       1        1  21.879694
1     -0.071944 -0.062903       2        1  14.926169
2     -0.104341  0.071810       3        1  14.338291
5     -0.018550 -0.045408       4        1  13.470085
6      0.022263  0.077690       5        1   9.733492
3     -0.091289  0.092859       6        1   7.618577
4     -0.217418 -0.198273       7        1   6.491327
7      0.201229  0.204804       8        1   6.294791
8      0.342574 -0.207945       9        1   5.247564, topic_info=               Term          Freq         Total Category  logprob  loglift
12             last   9108.000000   9108.000000  Default  30.0000  30.0000
82             long   9844.000000   9844.000000  Default  29.0000  29.0000
38            apply  11579.000000  11579.000000  Default  28.0000  28.0000
192        coverage   8362.000000   8362.000000  Default  27.0000  27.0000
64              use  37399.000000  37399.000000  Default  26.0000  26.0000
50             look  24260.000000  24260.000000  Default  25.0000  25.0000
62            smell   8271.000000   8271.000000  Default  24.0000  24.0000
77             star   5488.000000   5488.000000  Default  23.0000  23.0000
120            lash   6290.000000   6290.000000  Default  22.0000  22.0000
56               go  15428.000000  15428.000000  Default  21.0000  21.0000
34             oily   6677.000000   6677.000000  Default  20.0000  20.0000
4               day   9649.000000   9649.000000  Default  19.0000  19.0000
119           shade  10062.000000  10062.000000  Default  18.0000  18.0000
9              good  23361.000000  23361.000000  Default  17.0000  17.0000
46             feel  18808.000000  18808.000000  Default  16.0000  16.0000
20          product  42062.000000  42062.000000  Default  15.0000  15.0000
115          sample   8329.000000   8329.000000  Default  14.0000  14.0000
138          review   4723.000000   4723.000000  Default  13.0000  13.0000
94             dark   6298.000000   6298.000000  Default  12.0000  12.0000
102             see   6552.000000   6552.000000  Default  11.0000  11.0000
40             easy   4804.000000   4804.000000  Default  10.0000  10.0000
235            stay   5994.000000   5994.000000  Default   9.0000   9.0000
325           price   5154.000000   5154.000000  Default   8.0000   8.0000
125            wear   9273.000000   9273.000000  Default   7.0000   7.0000
44              dry   4125.000000   4125.000000  Default   6.0000   6.0000
39            color   6255.000000   6255.000000  Default   5.0000   5.0000
52             nice  15426.000000  15426.000000  Default   4.0000   4.0000
87          receive   5990.000000   5990.000000  Default   3.0000   3.0000
501          smooth   4718.000000   4718.000000  Default   2.0000   2.0000
127      difference   3178.000000   3178.000000  Default   1.0000   1.0000
39            color   6254.856445   6255.663086   Topic1  -4.0152   1.5195
616        lipstick   4169.349121   4170.155762   Topic1  -4.4208   1.5194
786         pigment   3450.402832   3451.209961   Topic1  -4.6101   1.5194
169            pink   3440.941162   3441.748291   Topic1  -4.6129   1.5194
117          shadow   3080.158936   3080.966064   Topic1  -4.7236   1.5193
348          bright   2362.371338   2363.178467   Topic1  -4.9889   1.5193
372           stain   1368.467896   1369.274780   Topic1  -5.5349   1.5190
309       highlight   1360.572510   1361.379395   Topic1  -5.5407   1.5190
366           sheer   1314.925659   1315.732544   Topic1  -5.5748   1.5190
614          colour   1259.774658   1260.581543   Topic1  -5.6177   1.5190
675          subtle   1135.400635   1136.207520   Topic1  -5.7216   1.5189
302         neutral   1034.503174   1035.310059   Topic1  -5.8147   1.5188
694      complexion    920.069031    920.876160   Topic1  -5.9319   1.5187
950            nude    789.255615    790.062622   Topic1  -6.08

In [160]:
display(neutral_lda.cv_results)

,Topics,Alpha,Beta,Coherence
12,9,asymmetric,0.1,0.494624
13,9,asymmetric,symmetric,0.494306
11,9,symmetric,symmetric,0.493490
8,9,0.1,0.1,0.492105
10,9,symmetric,0.1,0.489150
14,9,5.55556,0.1,0.488979
15,9,5.55556,symmetric,0.484942
9,9,0.1,symmetric,0.483010
5,3,asymmetric,symmetric,0.423331
4,3,asymmetric,0.1,0.421154


In [161]:
neutral_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062524  0.067366       1        1  21.891186
1     -0.071944 -0.062903       2        1  14.926285
2     -0.104341  0.071810       3        1  14.331020
5     -0.018550 -0.045408       4        1  13.472164
6      0.022263  0.077690       5        1   9.733612
3     -0.091289  0.092859       6        1   7.614524
4     -0.217418 -0.198273       7        1   6.491765
7      0.201229  0.204804       8        1   6.294011
8      0.342574 -0.207945       9        1   5.245436, topic_info=               Term          Freq         Total Category  logprob  loglift
12             last   9104.000000   9104.000000  Default  30.0000  30.0000
82             long   9840.000000   9840.000000  Default  29.0000  29.0000
38            apply  11578.000000  11578.000000  Default  28.0000  28.0000
192        coverage   8357.000000   8357.000000  Default  27.0000  27.0000
64              use  37401.000000  37401.000000  Default  26.0000  26.0000
50             look  24268.000000  24268.000000  Default  25.0000  25.0000
62            smell   8267.000000   8267.000000  Default  24.0000  24.0000
77             star   5488.000000   5488.000000  Default  23.0000  23.0000
120            lash   6290.000000   6290.000000  Default  22.0000  22.0000
56               go  15429.000000  15429.000000  Default  21.0000  21.0000
34             oily   6678.000000   6678.000000  Default  20.0000  20.0000
4               day   9648.000000   9648.000000  Default  19.0000  19.0000
119           shade  10065.000000  10065.000000  Default  18.0000  18.0000
9              good  23357.000000  23357.000000  Default  17.0000  17.0000
46             feel  18803.000000  18803.000000  Default  16.0000  16.0000
20          product  42060.000000  42060.000000  Default  15.0000  15.0000
115          sample   8330.000000   8330.000000  Default  14.0000  14.0000
138          review   4722.000000   4722.000000  Default  13.0000  13.0000
94             dark   6298.000000   6298.000000  Default  12.0000  12.0000
102             see   6553.000000   6553.000000  Default  11.0000  11.0000
40             easy   4804.000000   4804.000000  Default  10.0000  10.0000
235            stay   5993.000000   5993.000000  Default   9.0000   9.0000
325           price   5152.000000   5152.000000  Default   8.0000   8.0000
125            wear   9274.000000   9274.000000  Default   7.0000   7.0000
44              dry   4125.000000   4125.000000  Default   6.0000   6.0000
39            color   6258.000000   6258.000000  Default   5.0000   5.0000
52             nice  15428.000000  15428.000000  Default   4.0000   4.0000
87          receive   5991.000000   5991.000000  Default   3.0000   3.0000
501          smooth   4718.000000   4718.000000  Default   2.0000   2.0000
127      difference   3178.000000   3178.000000  Default   1.0000   1.0000
39            color   6258.137695   6258.944336   Topic1  -4.0152   1.5190
616        lipstick   4171.536133   4172.342773   Topic1  -4.4208   1.5189
786         pigment   3452.213135   3453.020264   Topic1  -4.6101   1.5189
169            pink   3442.746338   3443.553223   Topic1  -4.6129   1.5189
117          shadow   3081.774902   3082.582031   Topic1  -4.7236   1.5188
348          bright   2363.610596   2364.417480   Topic1  -4.9889   1.5187
372           stain   1369.185791   1369.992554   Topic1  -5.5349   1.5185
309       highlight   1361.286377   1362.093262   Topic1  -5.5407   1.5185
366           sheer   1315.615479   1316.422363   Topic1  -5.5748   1.5185
614          colour   1260.435547   1261.242432   Topic1  -5.6177   1.5184
675          subtle   1135.996216   1136.802979   Topic1  -5.7216   1.5184
302         neutral   1035.045898   1035.852783   Topic1  -5.8147   1.5183
694      complexion    920.551697    921.358643   Topic1  -5.9319   1.5182
950            nude    789.669617    790.476440   Topic1  -6.08

In [168]:
good_reviews = good_lda.get_docs_topics()

In [ ]:
neutral_reviews = neutral_lda.get_docs_topics()

In [ ]:
bad_reviews = bad_lda.get_docs_topics()

In [ ]:
good_reviews.to_pickle('good_reviews.pickle')
bad_reviews.to_pickle('bad_reviews.pickle')
neutral_reviews.to_pickle('neutral_reviews.pickle')

In [172]:
good_lda.cv_results

,Topics,Alpha,Beta,Coherence
29,9,5.55556,symmetric,0.529431
28,9,5.55556,0.1,0.528639
23,9,asymmetric,symmetric,0.514659
12,6,asymmetric,0.1,0.514239
13,6,asymmetric,symmetric,0.506266
21,9,symmetric,symmetric,0.506242
22,9,asymmetric,0.1,0.504448
17,6,8.33333,symmetric,0.502483
19,9,0.1,symmetric,0.500803
16,6,8.33333,0.1,0.497481


In [206]:
good_reviews_3 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='good_reviews')
bad_reviews_3 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='bad_reviews')
neutral_reviews_3 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta='symmetric', name='neutral_reviews')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.053791 -0.190983       1        1  53.823143
1     -0.161852  0.136313       2        1  30.715141
2      0.215643  0.054670       3        1  15.461715, topic_info=               Term          Freq         Total Category  logprob  loglift
7              love  44638.000000  44638.000000  Default  30.0000  30.0000
9               use  40521.000000  40521.000000  Default  29.0000  29.0000
4              easy   9714.000000   9714.000000  Default  28.0000  28.0000
143           apply  10078.000000  10078.000000  Default  27.0000  27.0000
151            lash   6589.000000   6589.000000  Default  26.0000  26.0000
64             look  25918.000000  25918.000000  Default  25.0000  25.0000
286        eyeliner   3785.000000   3785.000000  Default  24.0000  24.0000
76             brow   3747.000000   3747.000000  Default  23.0000  23.0000
78             long  12440.000000  12440.000000  Default  22.0000  22.0000
291            stay   7168.000000   7168.000000  Default  21.0000  21.0000
57             ever   5631.000000   5631.000000  Default  20.0000  20.0000
218            nice  10830.000000  10830.000000  Default  19.0000  19.0000
102            make  17320.000000  17320.000000  Default  18.0000  18.0000
10             year   5280.000000   5280.000000  Default  17.0000  17.0000
355            oily   5254.000000   5254.000000  Default  16.0000  16.0000
517           brush   2956.000000   2956.000000  Default  15.0000  15.0000
3               day  14594.000000  14594.000000  Default  14.0000  14.0000
67             work  12821.000000  12821.000000  Default  13.0000  13.0000
318           shade   7247.000000   7247.000000  Default  12.0000  12.0000
351        coverage   8151.000000   8151.000000  Default  11.0000  11.0000
6              last  11695.000000  11695.000000  Default  10.0000  10.0000
421            line   3831.000000   3831.000000  Default   9.0000   9.0000
65          perfect  10734.000000  10734.000000  Default   8.0000   8.0000
79          natural   6704.000000   6704.000000  Default   7.0000   7.0000
56            cover   3375.000000   3375.000000  Default   6.0000   6.0000
38             star   6407.000000   6407.000000  Default   5.0000   5.0000
227              go  17997.000000  17997.000000  Default   4.0000   4.0000
546            hard   2008.000000   2008.000000  Default   3.0000   3.0000
538            help   4293.000000   4293.000000  Default   2.0000   2.0000
235          easily   3095.000000   3095.000000  Default   1.0000   1.0000
318           shade   7247.350098   7247.560547   Topic1  -4.6934   0.6194
38             star   6407.204102   6407.414551   Topic1  -4.8166   0.6194
13        beautiful   5683.886719   5684.097168   Topic1  -4.9364   0.6194
48            color   5684.080078   5684.290527   Topic1  -4.9363   0.6194
34          receive   4667.147949   4667.358398   Topic1  -5.1335   0.6194
229        lipstick   4217.097168   4217.307617   Topic1  -5.2349   0.6194
369            tone   3788.802002   3789.012939   Topic1  -5.3420   0.6194
420            glow   3268.909668   3269.120605   Topic1  -5.4896   0.6194
337          finish   3148.696045   3148.907227   Topic1  -5.5270   0.6194
242         pigment   3134.278320   3134.489990   Topic1  -5.5316   0.6194
47          collect   2726.441406   2726.652100   Topic1  -5.6710   0.6194
50        promotion   2721.930176   2722.140869   Topic1  -5.6727   0.6194
526          shadow   2545.079834   2545.290771   Topic1  -5.7399   0.6194
219            pink   2282.136475   2282.347168   Topic1  -5.8489   0.6194
273         quality   1863.498047   1863.709106   Topic1  -6.0516   0.6194
52           sticky   1784.572266   1784.782959   Topic1  -6.0948   0.6193
395          creamy   1605.709839   1605.920532   Topic1  -6.2005   0.6193
342       packaging   1510.368530   1510.583374   Topic1  -6.2617   0.6193
529         

426.28794836997986


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.154094  0.125229       1        1  42.475506
1     -0.061503 -0.167073       2        1  39.455536
2      0.215598  0.041844       3        1  18.068960, topic_info=            Term          Freq         Total Category  logprob  loglift
19           use  36501.000000  36501.000000  Default  30.0000  30.0000
77         break   7209.000000   7209.000000  Default  29.0000  29.0000
190          day   8977.000000   8977.000000  Default  28.0000  28.0000
75         smell   9412.000000   9412.000000  Default  27.0000  27.0000
204        apply  10840.000000  10840.000000  Default  26.0000  26.0000
473    sensitive   3792.000000   3792.000000  Default  25.0000  25.0000
98          look  24282.000000  24282.000000  Default  24.0000  24.0000
209         time  10937.000000  10937.000000  Default  23.0000  23.0000
112         make  19122.000000  19122.000000  Default  22.0000  22.0000
102         oily   6790.000000   6790.000000  Default  21.0000  21.0000
72         money   6574.000000   6574.000000  Default  20.0000  20.0000
464         burn   3248.000000   3248.000000  Default  19.0000  19.0000
430         week   3187.000000   3187.000000  Default  18.0000  18.0000
59         shade   6152.000000   6152.000000  Default  17.0000  17.0000
68          lash   5647.000000   5647.000000  Default  16.0000  16.0000
162         star   6072.000000   6072.000000  Default  15.0000  15.0000
380     eyeliner   2894.000000   2894.000000  Default  14.0000  14.0000
211      receive   5767.000000   5767.000000  Default  13.0000  13.0000
6         sample   5338.000000   5338.000000  Default  12.0000  12.0000
21           bad  10174.000000  10174.000000  Default  11.0000  11.0000
152       smudge   2553.000000   2553.000000  Default  10.0000  10.0000
450        start   3506.000000   3506.000000  Default   9.0000   9.0000
205        first   6826.000000   6826.000000  Default   8.0000   8.0000
582        cause   2533.000000   2533.000000  Default   7.0000   7.0000
820        never   6109.000000   6109.000000  Default   6.0000   6.0000
11          hard   4614.000000   4614.000000  Default   5.0000   5.0000
201         hour   5425.000000   5425.000000  Default   4.0000   4.0000
157        waste   4203.000000   4203.000000  Default   3.0000   3.0000
107         feel  13447.000000  13447.000000  Default   2.0000   2.0000
110        thick   4046.000000   4046.000000  Default   1.0000   1.0000
75         smell   9411.909180   9412.113281   Topic1  -4.2456   0.8562
59         shade   6152.770996   6152.976074   Topic1  -4.6706   0.8562
211      receive   5767.588379   5767.793457   Topic1  -4.7353   0.8562
6         sample   5338.323242   5338.528320   Topic1  -4.8126   0.8562
371          old   4035.549072   4035.753662   Topic1  -5.0924   0.8562
210        order   3909.591064   3909.795654   Topic1  -5.1241   0.8562
167        color   3909.355713   3909.560303   Topic1  -5.1242   0.8562
270         open   3017.105713   3017.310303   Topic1  -5.3832   0.8562
53      lipstick   2963.790771   2963.995361   Topic1  -5.4011   0.8562
350         size   2731.655762   2731.860352   Topic1  -5.4826   0.8562
406         pink   2680.856689   2681.061279   Topic1  -5.5014   0.8562
161         tone   2580.857422   2581.062012   Topic1  -5.5394   0.8562
494          box   2563.746338   2563.950928   Topic1  -5.5461   0.8562
480    packaging   2559.114990   2559.319580   Topic1  -5.5479   0.8562
36       quality   2330.893311   2331.097900   Topic1  -5.6413   0.8562
217      package   2211.032227   2211.236816   Topic1  -5.6941   0.8562
269         item   2207.279785   2207.484375   Topic1  -5.6958   0.8561
253      pigment   2202.155518   2202.360107   Topic1  -5.6981   0.8561
447        scent   1985.650879   1985.855713   Topic1  -5.8016   0.8561
86        shadow   1900.971680   1901.176514   Topic1  -5.8452   0.8561
544     origina

364.6052129268646


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.024071 -0.158298       1        1  54.370426
1     -0.179128  0.062046       2        1  24.545347
2      0.155057  0.096252       3        1  21.084225, topic_info=             Term          Freq         Total Category  logprob  loglift
154         shade   9849.000000   9849.000000  Default  30.0000  30.0000
213          feel  18528.000000  18528.000000  Default  29.0000  29.0000
75         sample   7820.000000   7820.000000  Default  28.0000  28.0000
364          lash   6156.000000   6156.000000  Default  27.0000  27.0000
112         color   6494.000000   6494.000000  Default  26.0000  26.0000
314          dark   6122.000000   6122.000000  Default  25.0000  25.0000
2            nice  15030.000000  15030.000000  Default  24.0000  24.0000
100          tone   4087.000000   4087.000000  Default  23.0000  23.0000
165         brush   3639.000000   3639.000000  Default  22.0000  22.0000
266      lipstick   3934.000000   3934.000000  Default  21.0000  21.0000
7             use  36972.000000  36972.000000  Default  20.0000  20.0000
285          easy   5089.000000   5089.000000  Default  19.0000  19.0000
855      eyeliner   3324.000000   3324.000000  Default  18.0000  18.0000
340         apply  11419.000000  11419.000000  Default  17.0000  17.0000
46           come   5328.000000   5328.000000  Default  16.0000  16.0000
3         pigment   3507.000000   3507.000000  Default  15.0000  15.0000
76         shadow   3346.000000   3346.000000  Default  14.0000  14.0000
256          pink   3345.000000   3345.000000  Default  13.0000  13.0000
47           love  17179.000000  17179.000000  Default  12.0000  12.0000
166      coverage   7602.000000   7602.000000  Default  11.0000  11.0000
113           get  14173.000000  14173.000000  Default  10.0000  10.0000
223           day   9752.000000   9752.000000  Default   9.0000   9.0000
287        smudge   2714.000000   2714.000000  Default   8.0000   8.0000
74        receive   5914.000000   5914.000000  Default   7.0000   7.0000
37           oily   6576.000000   6576.000000  Default   6.0000   6.0000
84           time   8541.000000   8541.000000  Default   5.0000   5.0000
370          hard   4619.000000   4619.000000  Default   4.0000   4.0000
365          long   9852.000000   9852.000000  Default   3.0000   3.0000
215         price   5396.000000   5396.000000  Default   2.0000   2.0000
51        formula   4161.000000   4161.000000  Default   1.0000   1.0000
166      coverage   7601.372070   7602.099121   Topic1  -4.7306   0.6093
37           oily   6575.396484   6576.105469   Topic1  -4.8756   0.6092
215         price   5395.481445   5396.300781   Topic1  -5.0733   0.6092
88     difference   2959.180664   2959.883057   Topic1  -5.6740   0.6091
275            ok   3230.406982   3231.272461   Topic1  -5.5863   0.6091
262          brow   2712.360352   2713.142334   Topic1  -5.7611   0.6091
414     sensitive   2343.024658   2343.740234   Topic1  -5.9075   0.6090
509         cover   2430.556641   2431.304443   Topic1  -5.8708   0.6090
251         heavy   2417.333008   2418.079102   Topic1  -5.8762   0.6090
41           pore   2185.735840   2186.426514   Topic1  -5.9770   0.6090
741     concealer   2237.710205   2238.457275   Topic1  -5.9535   0.6090
1089      special   1951.781128   1952.565674   Topic1  -6.0902   0.6089
318        normal   1380.436401   1381.190186   Topic1  -6.4365   0.6088
474           spf   1286.028320   1286.738403   Topic1  -6.5074   0.6088
922         shiny   1348.772583   1349.529541   Topic1  -6.4597   0.6088
420     sunscreen   1205.487671   1206.196777   Topic1  -6.5720   0.6088
52         greasy   1070.601929   1071.298096   Topic1  -6.6907   0.6087
1091         spot   1153.626099   1154.377808   Topic1  -6.6160   0.6087
200       average   1234.712280   1235.525879   Topic1  -6.5481   0.6087
108       purpose   1195.539673   119

330.20173597335815


In [177]:
int_start=time.time()
# Inputs
tokens_good = random.sample(good_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_good = corpora.Dictionary(tokens_good)
# Term Document Frequency
corpus_good = [id2word_good.doc2bow(text) for text in tokens_good]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_good = gensim.models.LdaMulticore(corpus=corpus_good,
                                       id2word=id2word_good,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)
LDAvis_prepared

8.354961156845093
180.80599331855774


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062634 -0.160915       1        1  54.170780
2     -0.184087  0.115550       2        1  25.815531
1      0.246721  0.045365       3        1  20.013683, topic_info=               Term          Freq         Total Category  logprob  loglift
50          product  37768.000000  37768.000000  Default  30.0000  30.0000
17             love  45583.000000  45583.000000  Default  29.0000  29.0000
46             good  23070.000000  23070.000000  Default  28.0000  28.0000
26              use  39675.000000  39675.000000  Default  27.0000  27.0000
284            star   6688.000000   6688.000000  Default  26.0000  26.0000
52            shade   7152.000000   7152.000000  Default  25.0000  25.0000
12             feel  15812.000000  15812.000000  Default  24.0000  24.0000
16             look  25950.000000  25950.000000  Default  23.0000  23.0000
175            ever   5543.000000   5543.000000  Default  22.0000  22.0000
170             buy  11602.000000  11602.000000  Default  21.0000  21.0000
88             long  12283.000000  12283.000000  Default  20.0000  20.0000
443            lash   6211.000000   6211.000000  Default  19.0000  19.0000
98            great  31914.000000  31914.000000  Default  18.0000  18.0000
255           price   4708.000000   4708.000000  Default  17.0000  17.0000
83             work  12854.000000  12854.000000  Default  16.0000  16.0000
10            color   5501.000000   5501.000000  Default  15.0000  15.0000
8         beautiful   5218.000000   5218.000000  Default  14.0000  14.0000
30         favorite   5839.000000   5839.000000  Default  13.0000  13.0000
118            year   5220.000000   5220.000000  Default  12.0000  12.0000
128         receive   4525.000000   4525.000000  Default  11.0000  11.0000
41              try  12836.000000  12836.000000  Default  10.0000  10.0000
28            brand   3641.000000   3641.000000  Default   9.0000   9.0000
63         eyeliner   3562.000000   3562.000000  Default   8.0000   8.0000
85           sample   4953.000000   4953.000000  Default   7.0000   7.0000
243        lipstick   4155.000000   4155.000000  Default   6.0000   6.0000
70           really  17547.000000  17547.000000  Default   5.0000   5.0000
131        coverage   8373.000000   8373.000000  Default   4.0000   4.0000
417            part   3704.000000   3704.000000  Default   3.0000   3.0000
82             wear   9483.000000   9483.000000  Default   2.0000   2.0000
62             easy   9453.000000   9453.000000  Default   1.0000   1.0000
131        coverage   8372.796875   8373.002930   Topic1  -4.5553   0.6130
124           smell   6904.399414   6904.605957   Topic1  -4.7481   0.6130
315            oily   5048.371582   5048.578125   Topic1  -5.0612   0.6130
86             keep   4533.119141   4533.325684   Topic1  -5.1688   0.6130
31             help   4189.308594   4189.515137   Topic1  -5.2477   0.6130
105            brow   3290.972656   3291.178711   Topic1  -5.4891   0.6130
304           leave   3218.337402   3218.543457   Topic1  -5.5114   0.6130
58            cover   3024.253418   3024.459473   Topic1  -5.5736   0.6130
277       concealer   3012.024170   3012.230469   Topic1  -5.5776   0.6130
38        sensitive   2979.345215   2979.551270   Topic1  -5.5886   0.6130
452      moisturize   2631.087158   2631.293213   Topic1  -5.7129   0.6130
12             feel  15810.765625  15812.018555   Topic1  -3.9196   0.6129
107           clean   2368.465088   2368.671143   Topic1  -5.8180   0.6129
319           scent   2234.542236   2234.748291   Topic1  -5.8762   0.6129
165            pore   2224.448975   2224.655029   Topic1  -5.8807   0.6129
700           heavy   2208.018066   2208.224121   Topic1  -5.8882   0.6129
489            wash   1987.999878   1988.205933   Topic1  -5.9931   0.6129
384           start   1921.057373   1921.263428   Topic1  -6.0274   0.6129
386         

In [ ]:
pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)

In [178]:
bad_lda.cv_results

,Topics,Alpha,Beta,Coherence
10,9,symmetric,0.1,0.543806
13,9,asymmetric,symmetric,0.539759
9,9,0.1,symmetric,0.538955
12,9,asymmetric,0.1,0.538672
15,9,5.55556,symmetric,0.534416
14,9,5.55556,0.1,0.531963
11,9,symmetric,symmetric,0.531838
8,9,0.1,0.1,0.524311
4,3,asymmetric,0.1,0.479731
2,3,symmetric,0.1,0.479557


In [180]:
int_start=time.time()
# Inputs
tokens_bad = random.sample(bad_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_bad = corpora.Dictionary(tokens_bad)
# Term Document Frequency
corpus_bad = [id2word_bad.doc2bow(text) for text in tokens_bad]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_bad = gensim.models.LdaMulticore(corpus=corpus_bad,
                                       id2word=id2word_bad,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_bad, corpus_bad, id2word_bad)
LDAvis_prepared

5.881763219833374
180.23869347572327


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.066909 -0.196487       1        1  62.146614
1     -0.234042  0.049088       2        1  20.105894
2      0.167133  0.147399       3        1  17.747486, topic_info=              Term          Freq         Total Category  logprob  loglift
62         product  44627.000000  44627.000000  Default  30.0000  30.0000
14             use  36360.000000  36360.000000  Default  29.0000  29.0000
10           smell   9853.000000   9853.000000  Default  28.0000  28.0000
185           look  24007.000000  24007.000000  Default  27.0000  27.0000
72           money   6562.000000   6562.000000  Default  26.0000  26.0000
237           star   6245.000000   6245.000000  Default  25.0000  25.0000
294        receive   5622.000000   5622.000000  Default  24.0000  24.0000
81             buy  16215.000000  16215.000000  Default  23.0000  23.0000
267          break   7176.000000   7176.000000  Default  22.0000  22.0000
61             old   4032.000000   4032.000000  Default  21.0000  21.0000
43            make  18799.000000  18799.000000  Default  20.0000  20.0000
361          waste   4310.000000   4310.000000  Default  19.0000  19.0000
137           come   6885.000000   6885.000000  Default  18.0000  18.0000
16            year   3717.000000   3717.000000  Default  17.0000  17.0000
281         bottle   3885.000000   3885.000000  Default  16.0000  16.0000
581      sensitive   3525.000000   3525.000000  Default  15.0000  15.0000
192         sample   5215.000000   5215.000000  Default  14.0000  14.0000
141            day   9142.000000   9142.000000  Default  13.0000  13.0000
139          order   3665.000000   3665.000000  Default  12.0000  12.0000
17             bad  10089.000000  10089.000000  Default  11.0000  11.0000
77            burn   2928.000000   2928.000000  Default  10.0000  10.0000
52            tube   3096.000000   3096.000000  Default   9.0000   9.0000
80          really  15005.000000  15005.000000  Default   8.0000   8.0000
90           apply  10426.000000  10426.000000  Default   7.0000   7.0000
121           open   2998.000000   2998.000000  Default   6.0000   6.0000
412         change   2707.000000   2707.000000  Default   5.0000   5.0000
314          small   2964.000000   2964.000000  Default   4.0000   4.0000
332          cause   2527.000000   2527.000000  Default   3.0000   3.0000
831           size   2716.000000   2716.000000  Default   2.0000   2.0000
203        formula   4899.000000   4899.000000  Default   1.0000   1.0000
185           look  24007.570312  24007.777344   Topic1  -3.6866   0.4757
37            wear   7093.508789   7093.716797   Topic1  -4.9058   0.4756
451          shade   6020.921387   6021.129395   Topic1  -5.0697   0.4756
428           lash   5866.080078   5866.288086   Topic1  -5.0958   0.4756
28        coverage   5286.073730   5286.281738   Topic1  -5.1999   0.4756
357           nice   4855.183105   4855.391113   Topic1  -5.2849   0.4756
153           dark   4372.194824   4372.402832   Topic1  -5.3897   0.4756
232           stay   4039.636719   4039.844238   Topic1  -5.4688   0.4756
164          thick   3978.817383   3979.024902   Topic1  -5.4840   0.4756
528          color   3892.905762   3893.113281   Topic1  -5.5058   0.4756
426         sticky   3624.106445   3624.313965   Topic1  -5.5774   0.4756
408          super   3852.172119   3852.393555   Topic1  -5.5163   0.4756
685          blend   3123.031982   3123.239502   Topic1  -5.7262   0.4756
78             fan   3100.009521   3100.217041   Topic1  -5.7336   0.4756
158           tone   2671.843262   2672.050781   Topic1  -5.8822   0.4756
318         smudge   2638.359619   2638.567139   Topic1  -5.8948   0.4756
124           pink   2558.913574   2559.121094   Topic1  -5.9254   0.4756
74             set   2464.928223   2465.135742   Topic1  -5.9628   0.4756
257            end   2451.462891   2451.670410   Topic1  -5.9

In [181]:
int_start=time.time()
# Inputs
tokens_neutral = random.sample(neutral_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_neutral = corpora.Dictionary(tokens_neutral)
# Term Document Frequency
corpus_neutral = [id2word_neutral.doc2bow(text) for text in tokens_neutral]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_neutral = gensim.models.LdaMulticore(corpus=corpus_neutral,
                                       id2word=id2word_neutral,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 'symmetric',
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_neutral, corpus_neutral, id2word_neutral)
LDAvis_prepared

9.832677841186523
184.56911611557007


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.014317 -0.157837       1        1  54.757328
1     -0.176150  0.088164       2        1  23.077860
2      0.190467  0.069673       3        1  22.164816, topic_info=              Term          Freq         Total Category  logprob  loglift
97           shade  10011.000000  10011.000000  Default  30.0000  30.0000
7            smell   8207.000000   8207.000000  Default  29.0000  29.0000
115         sample   7638.000000   7638.000000  Default  28.0000  28.0000
170          color   6305.000000   6305.000000  Default  27.0000  27.0000
13            star   5469.000000   5469.000000  Default  26.0000  26.0000
3             nice  15261.000000  15261.000000  Default  25.0000  25.0000
56            look  23833.000000  23833.000000  Default  24.0000  24.0000
35             use  36141.000000  36141.000000  Default  23.0000  23.0000
75           apply  11157.000000  11157.000000  Default  22.0000  22.0000
786           tone   4308.000000   4308.000000  Default  21.0000  21.0000
277       lipstick   4260.000000   4260.000000  Default  20.0000  20.0000
88            dark   6125.000000   6125.000000  Default  19.0000  19.0000
55            long   9746.000000   9746.000000  Default  18.0000  18.0000
584           size   3900.000000   3900.000000  Default  17.0000  17.0000
54            last   8982.000000   8982.000000  Default  16.0000  16.0000
407           pink   3606.000000   3606.000000  Default  15.0000  15.0000
251          break   3382.000000   3382.000000  Default  14.0000  14.0000
214        pigment   3391.000000   3391.000000  Default  13.0000  13.0000
234         pretty   8324.000000   8324.000000  Default  12.0000  12.0000
50        coverage   7921.000000   7921.000000  Default  11.0000  11.0000
43         receive   5790.000000   5790.000000  Default  10.0000  10.0000
33         product  41180.000000  41180.000000  Default   9.0000   9.0000
529         shadow   2937.000000   2937.000000  Default   8.0000   8.0000
58            oily   6479.000000   6479.000000  Default   7.0000   7.0000
228           brow   2632.000000   2632.000000  Default   6.0000   6.0000
65            lash   6319.000000   6319.000000  Default   5.0000   5.0000
218         bottle   2418.000000   2418.000000  Default   4.0000   4.0000
160      sensitive   2417.000000   2417.000000  Default   3.0000   3.0000
77             day   9465.000000   9465.000000  Default   2.0000   2.0000
98            stay   5688.000000   5688.000000  Default   1.0000   1.0000
75           apply  11156.171875  11157.007812   Topic1  -4.3504   0.6022
55            long   9745.862305   9746.626953   Topic1  -4.4856   0.6022
50        coverage   7920.268555   7921.008789   Topic1  -4.6930   0.6022
65            lash   6318.706543   6319.396484   Topic1  -4.9189   0.6021
58            oily   6479.060059   6479.784180   Topic1  -4.8938   0.6021
172           easy   4808.720703   4809.482422   Topic1  -5.1920   0.6021
68           thick   4569.613770   4570.364258   Topic1  -5.2430   0.6021
275           hour   3968.960938   3969.686035   Topic1  -5.3839   0.6021
423           line   3719.994385   3720.787354   Topic1  -5.4487   0.6020
290          brush   3573.090576   3573.856934   Topic1  -5.4890   0.6020
245       eyeliner   2938.839844   2939.543457   Topic1  -5.6844   0.6020
171         easily   3067.131348   3067.953369   Topic1  -5.6417   0.6020
402         smudge   2406.755127   2407.450195   Topic1  -5.8841   0.6020
151          heavy   2429.419434   2430.187744   Topic1  -5.8747   0.6019
219      concealer   2207.825195   2208.579102   Topic1  -5.9704   0.6019
98            stay   5686.668945   5688.659180   Topic1  -5.0243   0.6019
362          clump   1935.303345   1935.998535   Topic1  -6.1021   0.6019
54            last   8979.122070   8982.676758   Topic1  -4.5675   0.6019
508           thin   1742.676636   1743.422729   Topic1  -6.2

## NLP Preprocessing

In [ ]:
docs = pickle.load(open("docs.pickle", "rb"))

d = corpora.Dictionary(docs)

freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [75]:
freq[freq['freq']>500]

,idx,freq,token
115,114,1612289,product
8,24,1465876,use
41,44,1461919,love
114,111,1117876,great
14,20,1110963,skin
31,43,1043474,look
5,8,1008814,good
119,117,941979,color
47,45,771895,make
78,70,768706,go


## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    self.corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
# docs = pickle.load(open("docs.p", "rb"))
# id2word = pickle.load(open("id2word.p", "rb"))

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in docs]

# output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

# output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)